<a href="https://colab.research.google.com/github/sznajder/Notebooks/blob/master/DiHiggs_JPata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Log in to the server using SSH:
```bash
ssh -i /path/to/id_rsa imperium-skel-sm.hep.caltech.edu`.
```

First time only, create a software sandbox:

```bash
sudo singularity build --sandbox ~/my-sandbox /bigdata/shared/Software/singularity/ibanks/edge.simg
```

Run a jupyter notebook:
```bash
/bigdata/shared/Software/jupyter/start_S.sh ~/my-sandbox
```
On your computer, open a browser at the given URL and go to your home directory. Create a new notebook with `New->Python3`.

Run a shell in the sandbox:
```bash
/bigdata/shared/Software/jupyter/start.sh ~/my-sandbox
```

In case you need to install any packages in the sandbox:
```bash
sudo singularity shell --writable ~/my-sandbox
> pip3 install package
> yum install library
```

In [ ]:
import uproot

In [ ]:
fi = uproot.open("/mnt/hadoop/store/mc/RunIIAutumn18NanoAODv5/GluGluToHHTo2B2G_node_SM_TuneCP5_PSWeights_13TeV-madgraph-pythia8/NANOAODSIM/Nano1June2019_102X_upgrade2018_realistic_v19-v1/100000/0AE1DF70-D618-164A-BE18-36C083A75CE4.root")

In [ ]:
events = fi.get("Events")

In [ ]:
for k in events.keys():
    k  = str(k, "ascii")
    if k.startswith("GenPart_"):
        print(k)

GenPart_eta
GenPart_mass
GenPart_phi
GenPart_pt
GenPart_genPartIdxMother
GenPart_pdgId
GenPart_status
GenPart_statusFlags


Load the data to memory.

In [ ]:
data = events.arrays([b"GenPart_pt", b"GenPart_eta", b"GenPart_pdgId", b"GenPart_status", b"GenPart_genPartIdxMother"])
data = {str(k, 'ascii'): v for k, v in data.items()}

In [ ]:
data["GenPart_pt"]

<JaggedArray [[0.0 0.0 105.0 ... 10.625 18.0 10.40625] [0.0 0.0 241.5 ... 15.4375 97.75 84.0] [0.0 0.0 153.5 ... 0.2109375 0.09887695 31.1875] ... [0.0 0.0 249.0 ... 2.484375 11.59375 63.5] [0.0 0.0 184.0 ... 0.103271484 8.875 46.25] [0.0 0.0 19.0625 ... 2.125 1.2890625 13.34375]] at 0x7f28b48a5748>

We have 400k events.

In [ ]:
len(data["GenPart_pt"])

400000

Each event has multiple particles, in total we have close to 20M particles.

In [ ]:
len(data["GenPart_pt"].content)

18757597

Let's print out a few events and the particle content in the events.

In [ ]:
for iev in range(2):
    ngp = data["GenPart_pt"].counts[iev]
    print("Event iev={0} has {1} generated particles".format(iev, ngp))
    for igp in range(ngp):
        print("  igp={0} pt={1} pdgId={2} status={3} idxmother={4}".format(
            igp,
            data["GenPart_pt"][iev][igp],
            data["GenPart_pdgId"][iev][igp],
            data["GenPart_status"][iev][igp],
            data["GenPart_genPartIdxMother"][iev][igp])
    )

Event iev=0 has 41 generated particles
  igp=0 pt=0.0 pdgId=21 status=21 idxmother=-1
  igp=1 pt=0.0 pdgId=21 status=21 idxmother=-1
  igp=2 pt=105.0 pdgId=25 status=22 idxmother=0
  igp=3 pt=105.0 pdgId=25 status=22 idxmother=0
  igp=4 pt=134.5 pdgId=25 status=44 idxmother=2
  igp=5 pt=104.5 pdgId=25 status=44 idxmother=3
  igp=6 pt=135.0 pdgId=25 status=44 idxmother=4
  igp=7 pt=105.0 pdgId=25 status=44 idxmother=5
  igp=8 pt=135.0 pdgId=25 status=44 idxmother=6
  igp=9 pt=106.0 pdgId=25 status=44 idxmother=7
  igp=10 pt=135.5 pdgId=25 status=44 idxmother=8
  igp=11 pt=101.25 pdgId=25 status=44 idxmother=9
  igp=12 pt=137.5 pdgId=25 status=44 idxmother=10
  igp=13 pt=101.25 pdgId=25 status=44 idxmother=11
  igp=14 pt=137.0 pdgId=25 status=44 idxmother=12
  igp=15 pt=100.75 pdgId=25 status=44 idxmother=13
  igp=16 pt=137.0 pdgId=25 status=44 idxmother=14
  igp=17 pt=101.25 pdgId=25 status=44 idxmother=15
  igp=18 pt=137.5 pdgId=25 status=62 idxmother=16
  igp=19 pt=101.25 pdgId=25 sta

## Task 0: directed graph of generated particles

Write a function that creates a directed graph for the generated particles for each event.
The input will be a list of particles with (idx, pdgId, idxmother). The output should be a graph, with nodes corresponding to the particles and edges corresponding to the (idx, idxmother) of each particle.

Visualizes the graphs for 3 events.

Use a library e.g. `networkx` (or suggest an alternative) to work with the graphs.

## Task 1: find all the stable final state particles

Write a function that will find the final b quarks (pdgId==+-5) and photons (pdgId==22) in each event. Final meaning that they are in the end of the decay chain, followed by other particles only.

Input: list of particles with (idx, pdgId, idxmother)
Output: indices of b1, b2, photon1, photon2 in the list